## Load data

In [18]:
import pandas as pd
import glob

In [19]:
# Where is the data: 
filenames = glob.glob("*.csv")

In [20]:
# Load the metadata 
# Only the stuff at the tope
df_meta = pd.read_csv(filenames[0], nrows=10, dtype=str, header=None, names=range(4))
df_meta = df_meta.rename(columns={0:'metadata_name_1', 1:'metadata_1', 2:'metadata_name_2', 3:'metadata_2'})

# There's actually two metadata tables here
df_meta_1 = df_meta[['metadata_name_1','metadata_1']]
df_meta_2 = df_meta[['metadata_name_2','metadata_2']].dropna(axis=0)

# We need to transpose the data into rows
df_meta_2 = df_meta_2.T

# Column headers from a row 
df_meta_2.columns = df_meta_2.iloc[0]

# Drop the row now we have the column headers
df_meta_2.drop(df_meta_2.iloc[0].name, inplace=True)

# Clean headers
df_meta_2 = df_meta_2.rename(columns=lambda x: x.replace(':',''))

# Do the same for metadata_1 
df_meta_1 = df_meta_1.T
df_meta_1.columns = df_meta_1.iloc[0]
df_meta_1.drop(df_meta_1.iloc[0].name, inplace=True)
df_meta_1 = df_meta_1.rename(columns=lambda x: x.replace(':',''))

In [21]:
df_meta_1

metadata_name_1,Data file,Sample name,Description,Sample amount,Instrument,Injection date,Acq. method,Analysis method,Last changed,Signal
metadata_1,C:\CDSProjects\Dingwall_group\Results\PD - TEM...,PD-007 - Q 10,NaN,0.000,1100LC,2019-08-26 14:46:04+01:00,PD - TEMPO Gradient.amx,*GC_LC Area Percent_DefaultMethod.pmx,2019-06-19 10:57:01+01:00,"VWD1A,Wavelength=210 nm"


In [22]:
df_meta_2

metadata_name_2,Sample type,Location,Injection,Injection volume,Acq. operator
metadata_2,Sample,30,1 of 1,2.000,SYSTEM


In [23]:
# Load the actual data 
df_data = pd.read_csv(filenames[0], skiprows=10)

# Remove the row with the na
df_data.dropna(axis=0, inplace=True, thresh=4)

In [24]:
df_data

,RT [min],Type,Width [min],Area,Height,Area%,Name
0,1.510,BB,3.0299,55230.0460,2747.4610,96.8653,NaN
1,5.116,BB,0.8348,221.6965,11.9836,0.3888,NaN
2,7.096,BB,1.0639,1565.6123,160.8382,2.7459,NaN


## Combine the datasets 

In [25]:
nrows = len(df_data)
print('There are {} rows in the data'.format(nrows))

# Make the metadata the right size
tmp_meta_1 = pd.concat([df_meta_1]*nrows, ignore_index=True)
tmp_meta_2 = pd.concat([df_meta_2]*nrows, ignore_index=True)

print('There are {} rows in metadata 1'.format(len(tmp_meta_1)))
print('There are {} rows in metadata 2'.format(len(tmp_meta_2)))

# Join metadata and data on the columns
df = pd.concat([df_data, tmp_meta_1, tmp_meta_2], axis=1)

print('There are {} rows in the joined data (df)'.format(len(df)))


There are 3 rows in the data
There are 3 rows in metadata 1
There are 3 rows in metadata 2
There are 3 rows in the joined data (df)


In [26]:
df

,RT [min],Type,Width [min],Area,Height,Area%,Name,Data file,Sample name,Description,...,Injection date,Acq. method,Analysis method,Last changed,Signal,Sample type,Location,Injection,Injection volume,Acq. operator
0,1.510,BB,3.0299,55230.0460,2747.4610,96.8653,NaN,C:\CDSProjects\Dingwall_group\Results\PD - TEM...,PD-007 - Q 10,NaN,...,2019-08-26 14:46:04+01:00,PD - TEMPO Gradient.amx,*GC_LC Area Percent_DefaultMethod.pmx,2019-06-19 10:57:01+01:00,"VWD1A,Wavelength=210 nm",Sample,30,1 of 1,2.000,SYSTEM
1,5.116,BB,0.8348,221.6965,11.9836,0.3888,NaN,C:\CDSProjects\Dingwall_group\Results\PD - TEM...,PD-007 - Q 10,NaN,...,2019-08-26 14:46:04+01:00,PD - TEMPO Gradient.amx,*GC_LC Area Percent_DefaultMethod.pmx,2019-06-19 10:57:01+01:00,"VWD1A,Wavelength=210 nm",Sample,30,1 of 1,2.000,SYSTEM
2,7.096,BB,1.0639,1565.6123,160.8382,2.7459,NaN,C:\CDSProjects\Dingwall_group\Results\PD - TEM...,PD-007 - Q 10,NaN,...,2019-08-26 14:46:04+01:00,PD - TEMPO Gradient.amx,*GC_LC Area Percent_DefaultMethod.pmx,2019-06-19 10:57:01+01:00,"VWD1A,Wavelength=210 nm",Sample,30,1 of 1,2.000,SYSTEM


## Do the above in a loop through our list of filenames

In [27]:
df_all = pd.DataFrame()

for file in filenames:
    print('I am processing {}'.format(file))
    
    # Load the metadata 
    # Only the stuff at the tope
    df_meta = pd.read_csv(file, nrows=10, dtype=str, header=None, names=range(4))
    df_meta = df_meta.rename(columns={0:'metadata_name_1', 1:'metadata_1', 2:'metadata_name_2', 3:'metadata_2'})

    # There's actually two metadata tables here
    df_meta_1 = df_meta[['metadata_name_1','metadata_1']]
    df_meta_2 = df_meta[['metadata_name_2','metadata_2']].dropna(axis=0)

    # We need to transpose the data into rows
    df_meta_2 = df_meta_2.T

    # Column headers from a row 
    df_meta_2.columns = df_meta_2.iloc[0]

    # Drop the row now we have the column headers
    df_meta_2.drop(df_meta_2.iloc[0].name, inplace=True)

    # Clean headers
    df_meta_2 = df_meta_2.rename(columns=lambda x: x.replace(':',''))

    # Do the same for metadata_1 
    df_meta_1 = df_meta_1.T
    df_meta_1.columns = df_meta_1.iloc[0]
    df_meta_1.drop(df_meta_1.iloc[0].name, inplace=True)
    df_meta_1 = df_meta_1.rename(columns=lambda x: x.replace(':',''))
    
    # Load the actual data 
    df_data = pd.read_csv(file, skiprows=10)

    # Remove the row with the na
    df_data.dropna(axis=0, inplace=True, thresh=4)
    
    nrows = len(df_data)
    print('There are {} rows in the data'.format(nrows))
    
    # Make the metadata the right size
    tmp_meta_1 = pd.concat([df_meta_1]*nrows, ignore_index=True)
    tmp_meta_2 = pd.concat([df_meta_2]*nrows, ignore_index=True)

    print('There are {} rows in metadata 1'.format(len(tmp_meta_1)))
    print('There are {} rows in metadata 2'.format(len(tmp_meta_2)))

    # Join metadata and data 
    df = pd.concat([df_data, tmp_meta_1, tmp_meta_2], axis=1)
    print('There are {} rows in the joined data (df)'.format(len(df)))

    # Join to the 
    df_all = pd.concat([df_all, df], axis=0)

df_all = df_all.reset_index(drop=True)

print('There are {} rows in the complete dataset'.format(len(df_all)))


I am processing PD-007 - Q 10_1.csv
There are 3 rows in the data
There are 3 rows in metadata 1
There are 3 rows in metadata 2
There are 3 rows in the joined data (df)
I am processing PD-007 - Q 1_1.csv
There are 4 rows in the data
There are 4 rows in metadata 1
There are 4 rows in metadata 2
There are 4 rows in the joined data (df)
I am processing PD-007 - Q 2_1.csv
There are 3 rows in the data
There are 3 rows in metadata 1
There are 3 rows in metadata 2
There are 3 rows in the joined data (df)
I am processing PD-007 - Q 3_1.csv
There are 3 rows in the data
There are 3 rows in metadata 1
There are 3 rows in metadata 2
There are 3 rows in the joined data (df)
I am processing PD-007 - Q 4_1.csv
There are 3 rows in the data
There are 3 rows in metadata 1
There are 3 rows in metadata 2
There are 3 rows in the joined data (df)
I am processing PD-007 - Q 5_1.csv
There are 3 rows in the data
There are 3 rows in metadata 1
There are 3 rows in metadata 2
There are 3 rows in the joined data (d

In [28]:
df_all.to_csv('final_data.csv', index=False)